# Modelos de prediccion
Vamos a probar distintos enfoques:
Enfoques de serie de tiempo:
* Suavizado exponencial #investigar

Enfoques Machine Learning:
* XGboost: Emiii
* LIGHT_GBM: Rugo
* Catboost: Vano

In [2]:
from DataLoader import importar_ventas,importar_cluster_stores
import xgboost as xgb
import pandas as pd

NameError: name 'importar_ventas' is not defined

In [ ]:
ventas=importar_ventas()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [36]:

FEATURES=['day','month','year', 'mean_price', 
          'cluster', 'cod_store_subgroup',
       'demand_lag_1', 'demand_lag_2', 'demand_lag_3', 'demand_lag_4','demand_lag_5', 'demand_lag_6', 'demand_lag_7']
TARGET=['demand']

test=ventas.groupby("cod_store_subgroup").tail(7)
train=ventas.drop(test.index)

X_train=train[FEATURES]
y_train=train[TARGET]
X_test=test[FEATURES]
y_test=test[TARGET]



In [37]:
from sklearn.metrics import r2_score

reg = xgb.XGBRegressor(
    n_estimators=1000,
    eval_metric=r2_score,
    eta=0.3,
    max_depth=3,
    early_stopping_rounds=100         
)

reg.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],   
    verbose=50                       
)

[0]	validation_0-rmse:3.03317	validation_0-r2_score:-0.03368
[50]	validation_0-rmse:2.96673	validation_0-r2_score:0.01111
[100]	validation_0-rmse:2.95905	validation_0-r2_score:0.01622
[115]	validation_0-rmse:2.95781	validation_0-r2_score:0.01704


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=100,
             enable_categorical=False, eta=0.3,
             eval_metric=<function r2_score at 0x00000175C21BB600>,
             feature_types=None, feature_weights=None, gamma=None,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=None, ...)

In [27]:
X_test

,day,month,year,mean_price,cluster,cod_store_subgroup,demand_lag_1,demand_lag_2,demand_lag_3,demand_lag_4,demand_lag_5,demand_lag_6,demand_lag_7
498,24,12,2023,35.430000,2,0,7.0,2.0,7.0,5.0,2.0,25.0,3.0
499,25,12,2023,29.100000,2,0,1.0,7.0,2.0,7.0,5.0,2.0,25.0
500,26,12,2023,37.120000,2,0,7.0,1.0,7.0,2.0,7.0,5.0,2.0
501,27,12,2023,30.723333,2,0,6.0,7.0,1.0,7.0,2.0,7.0,5.0
502,28,12,2023,37.990000,2,0,7.0,6.0,7.0,1.0,7.0,2.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5206788,25,12,2023,104.840000,4,11095,1.0,4.0,2.0,3.0,4.0,3.0,4.0
5206789,26,12,2023,91.200000,4,11095,2.0,1.0,4.0,2.0,3.0,4.0,3.0
5206790,27,12,2023,98.540000,4,11095,2.0,2.0,1.0,4.0,2.0,3.0,4.0
5206791,28,12,2023,98.923333,4,11095,5.0,2.0,2.0,1.0,4.0,2.0,3.0


In [42]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import PredefinedSplit



X_full=pd.concat([X_train,X_test])
y_full=pd.concat([y_train,y_test])

diferenciacion=[-1]*len(X_train)+[0]*len(X_test)
ps = PredefinedSplit(diferenciacion)


reg_grid=xgb.XGBRegressor()
param_grid = {'eta': [0.2,0.3,0.4],
    'max_depth': [2,3,6,8],
     'n_estimators': [50,100,150]}



grid_search = GridSearchCV(reg_grid, 
                           param_grid, 
                           cv=ps,
                           scoring='r2')
grid_search.fit(X_full,y_full)


GridSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0])),
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, feature_weights=None,
                                    gamma=None, grow_policy=None,
                                    i...
                                    learning_rate=None, max_bin=None,
                                    max_cat_threshold=None,
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None, ...),
             param_grid={'eta': [0.2, 0.3, 0.4], 'max_depth': [2, 3, 6, 8],
                         'n_estimators': [50, 100, 150]},
             scoring='r2')

In [51]:
pd.DataFrame(grid_search.cv_results_).sort_values(by="split0_test_score",ascending=False)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_eta,param_max_depth,param_n_estimators,params,split0_test_score,mean_test_score,std_test_score,rank_test_score
20,59.365036,0.0,0.129849,0.0,0.3,6,150,"{'eta': 0.3, 'max_depth': 6, 'n_estimators': 150}",0.033250,0.033250,0.0,1
19,31.956481,0.0,0.091918,0.0,0.3,6,100,"{'eta': 0.3, 'max_depth': 6, 'n_estimators': 100}",0.033115,0.033115,0.0,2
18,18.879518,0.0,0.053407,0.0,0.3,6,50,"{'eta': 0.3, 'max_depth': 6, 'n_estimators': 50}",0.031518,0.031518,0.0,3
8,46.787470,0.0,0.152122,0.0,0.2,6,150,"{'eta': 0.2, 'max_depth': 6, 'n_estimators': 150}",0.029645,0.029645,0.0,4
7,32.005171,0.0,0.078285,0.0,0.2,6,100,"{'eta': 0.2, 'max_depth': 6, 'n_estimators': 100}",0.029213,0.029213,0.0,5
6,18.901373,0.0,0.039175,0.0,0.2,6,50,"{'eta': 0.2, 'max_depth': 6, 'n_estimators': 50}",0.026106,0.026106,0.0,6
17,25.284546,0.0,0.060850,0.0,0.3,3,150,"{'eta': 0.3, 'max_depth': 3, 'n_estimators': 150}",0.023289,0.023289,0.0,7
31,30.699935,0.0,0.149576,0.0,0.4,6,100,"{'eta': 0.4, 'max_depth': 6, 'n_estimators': 100}",0.017198,0.017198,0.0,8
32,50.590065,0.0,0.107938,0.0,0.4,6,150,"{'eta': 0.4, 'max_depth': 6, 'n_estimators': 150}",0.017012,0.017012,0.0,9
16,17.362342,0.0,0.037312,0.0,0.3,3,100,"{'eta': 0.3, 'max_depth': 3, 'n_estimators': 100}",0.016201,0.016201,0.0,10


In [52]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import PredefinedSplit



X_full=pd.concat([X_train,X_test])
y_full=pd.concat([y_train,y_test])

diferenciacion=[-1]*len(X_train)+[0]*len(X_test)
ps = PredefinedSplit(diferenciacion)


reg_grid=xgb.XGBRegressor()
param_grid = {'eta': [0.3],
    'max_depth': [6],
     'n_estimators': [100]
    }



grid_search2 = GridSearchCV(reg_grid, 
                           param_grid, 
                           cv=ps,
                           scoring='r2')
grid_search2.fit(X_full,y_full)


GridSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0])),
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, feature_weights=None,
                                    gamma=None, grow_policy=None,
                                    i...
                                    learning_rate=None, max_bin=None,
                                    max_cat_threshold=None,
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None, ...),
             param_grid={'eta': [0.32, 0.35, 0.37], 'max_depth': [6, 7],
                         'n_estimators': [100, 150, 200, 250]},
             scoring='r2')

In [50]:
_full.isnull().sum()

demand    0
dtype: int64

In [53]:
pd.DataFrame(grid_search2.cv_results_).sort_values(by="split0_test_score",ascending=False)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_eta,param_max_depth,param_n_estimators,params,split0_test_score,mean_test_score,std_test_score,rank_test_score
11,64.972422,0.0,0.162240,0.0,0.35,6,250,"{'eta': 0.35, 'max_depth': 6, 'n_estimators': ...",0.033690,0.033690,0.0,1
19,67.307622,0.0,0.185594,0.0,0.37,6,250,"{'eta': 0.37, 'max_depth': 6, 'n_estimators': ...",0.032395,0.032395,0.0,2
8,26.112662,0.0,0.070285,0.0,0.35,6,100,"{'eta': 0.35, 'max_depth': 6, 'n_estimators': ...",0.032204,0.032204,0.0,3
10,57.557480,0.0,0.145150,0.0,0.35,6,200,"{'eta': 0.35, 'max_depth': 6, 'n_estimators': ...",0.032136,0.032136,0.0,4
9,37.514790,0.0,0.108431,0.0,0.35,6,150,"{'eta': 0.35, 'max_depth': 6, 'n_estimators': ...",0.032132,0.032132,0.0,5
17,48.622398,0.0,0.109699,0.0,0.37,6,150,"{'eta': 0.37, 'max_depth': 6, 'n_estimators': ...",0.031646,0.031646,0.0,6
16,39.857527,0.0,0.082253,0.0,0.37,6,100,"{'eta': 0.37, 'max_depth': 6, 'n_estimators': ...",0.031301,0.031301,0.0,7
18,58.700797,0.0,0.152795,0.0,0.37,6,200,"{'eta': 0.37, 'max_depth': 6, 'n_estimators': ...",0.031274,0.031274,0.0,8
3,71.232699,0.0,0.167347,0.0,0.32,6,250,"{'eta': 0.32, 'max_depth': 6, 'n_estimators': ...",0.024739,0.024739,0.0,9
1,42.043415,0.0,0.101100,0.0,0.32,6,150,"{'eta': 0.32, 'max_depth': 6, 'n_estimators': ...",0.023266,0.023266,0.0,10


In [57]:
grid_search.best_params_

{'eta': 0.3, 'max_depth': 6, 'n_estimators': 150}

In [59]:
reg=xgb.XGBRegressor(eta=0.3,max_depth=6,n_estimators=150)
reg.fit(X_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eta=0.3, eval_metric=None,
             feature_types=None, feature_weights=None, gamma=None,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=150, n_jobs=None, ...)

In [66]:
pd.DataFrame(reg.feature_importances_,index=reg.feature_names_in_,columns=["importance"]).sort_values(by="importance",
                                                                                                      ascending=False)

,importance
year,0.219976
month,0.172160
cluster,0.133384
mean_price,0.085870
demand_lag_1,0.070750
demand_lag_3,0.068313
demand_lag_2,0.060988
demand_lag_4,0.047780
demand_lag_5,0.039554
day,0.037860
